In [51]:
import numpy as np
import pandas as pd
import scipy.stats as st
import concurrent.futures

In [5]:
def plumed_to_pandas(filename):
    headers = pd.read_csv(filename,sep=' ',skipinitialspace=True, nrows=0).columns[2:]  
    df = pd.read_csv(filename,sep=' ',skipinitialspace=True, header=None,skiprows=1,names=headers,comment='#') 
    return df

In [ ]:
# 1D CASE: support derivatives and periodic CVs

def compute_fes_1d(cv_x,logweights,kbt,sigma,extent,grid_bin=100,compute_derivatives=False,periodic=False,compute_deltaF=False,deltaF_max=0.):
    #define grid
    gx_min,gx_max = extent
    grid_x=np.linspace(gx_min,gx_max,grid_bin)
    
    #compute probability
    max_prob=0
    prob=np.zeros(grid_bin)
    deriv=[]
    if compute_derivatives:
        deriv=np.zeros(grid_bin)
    basinA,basinB=0,0
    period=0
    if periodic:
        period=gx_max-gx_min
    if periodic and compute_derivatives:
        print('derivatives not implemented for periodic variables')
        return None
        
    for i in range(grid_bin):
        dx,arg2=0,0
        if periodic:
            dx=np.absolute(grid_x[i]-cv_x)
            arg2=(np.minimum(dx,period-dx)/sigma)**2
        else:
            dx=grid_x[i]-cv_x
            arg2=(dx/sigma)**2
        prob[i]=np.sum(np.exp(logweights)*np.exp(-0.5*arg2))
        if compute_derivatives:
            deriv[i]=np.sum(-dx/(sigma**2)*np.exp(logweights)*np.exp(-0.5*arg2))
        if prob[i]>max_prob:
            max_prob=prob[i]
        if compute_deltaF:
            if grid_x[i]<deltaF_max:
                basinA+=prob[i]
            else:
                basinB+=prob[i]

    #convert to fes
    fes=-kbt*np.log(prob/max_prob)
    
    if compute_derivatives and compute_deltaF:
        deriv=-kbt*deriv/prob
        return grid_x,fes,deriv,deltaF
    elif compute_deltaF:
        deltaF=kbt*np.log(basinA/basinB)
        return grid_x,fes,deltaF
    else:
        return grid_x,fes

# 2D case

def compute_fes_2d(cv_x,cv_y,logweights,kbt,sigma,extent,grid_bin=100):
    #define grid
    gx_min,gx_max,gy_min,gy_max = extent
    xx=np.linspace(gx_min,gx_max,grid_bin)
    yy=np.linspace(gy_min,gy_max,grid_bin)
    grid_x,grid_y=np.meshgrid(xx,yy)

    #retrieve_sigma
    if isinstance(sigma, list):
        sigma_x,sigma_y=sigma
    else:
        sigma_x=sigma
        sigma_y=sigma
        
    #compute probability
    max_prob=0
    prob=np.zeros((grid_bin,grid_bin))
    for i in range(grid_bin):
        for j in range(grid_bin):
            dx=np.absolute(grid_x[i,j]-cv_x)
            dy=np.absolute(grid_y[i,j]-cv_y)
            arg2=(dx/sigma_x)**2+(dy/sigma_y)**2
            prob[i,j]=np.sum(np.exp(logweights)*np.exp(-0.5*arg2)) 
            if prob[i,j]>max_prob:
                max_prob=prob[i,j]

    #convert to fes
    fes=-kbt*np.log(prob/max_prob)
    
    return fes

In [79]:
def compute_fes(colvar, collective_vars, bounds, num=100, bw_method = 0.05, kbt=2.5, logweights=None ):
    
    ndims = len(collective_vars)
    sigma = [bw_method for _ in range(ndims)]

    _1d_grid = [np.linspace(vmin, vmax, num) for (vmin, vmax) in bounds]
    meshgrids = np.meshgrid(*_1d_grid)
    coords = np.array([np.ravel(coord) for coord in meshgrids]).T

    #compute probability with KDE
    max_prob=0
    prob = np.zeros(coords.shape[0]) #np.zeros([num for _ in range(ndims)])

    for i,sample in enumerate(coords):
        arg2 = 0
        for j in range(ndims):
            dx_j = sample[j] - colvar[collective_vars[j]]
            arg2 += np.power(dx_j/sigma[j],2)
            if logweights:
                prob[i] = np.sum(np.exp(logweights)*np.exp(-0.5*arg2))
            else:
                prob[i] = np.sum(np.exp(-0.5*arg2))
            if prob[i] > max_prob:
                max_prob = prob[i]

    prob = np.reshape(prob,(num,)*ndims)

    #convert to fes
    fes=-kbt*np.log(prob/max_prob)

    return fes

collective_vars = ['tica1','tica2']
bounds = [(-1.,1.)]*len(collective_vars)

fes = compute_fes(colvar,collective_vars, bounds, num=11, kbt=2.8)


/tmp/ipykernel_562128/112209093.py:8: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  coords = np.vstack(map(np.ravel, meshgrids)).T


In [75]:
def _fes_gridpoint(sample,colvar,sigma,ndims,logweights):
    arg2 = 0
    for j in range(ndims):
        dx_j = sample[j] - colvar[collective_vars[j]]
        arg2 += np.power(dx_j/sigma[j],2)
        if logweights:
            p_i = np.sum(np.exp(logweights)*np.exp(-0.5*arg2))
        else:
            p_i = np.sum(np.exp(-0.5*arg2))
    return p_i

def compute_fes(colvar, collective_vars, bounds, num=100, bw_method = 0.05, kbt=2.5, logweights=None ):
    
    ndims = len(collective_vars)
    sigma = [bw_method for _ in range(ndims)]

    _1d_samples = [np.linspace(vmin, vmax, num) for (vmin, vmax) in bounds]
    meshgrids = np.meshgrid(*_1d_samples)
    sampled_positions = np.array([np.ravel(coord) for coord in meshgrids]).T

    #compute probability with KDE
    max_prob=0
    prob = np.zeros(sampled_positions.shape[0]) #np.zeros([num for _ in range(ndims)])

    for i,sample in enumerate(sampled_positions):
        prob[i] = _fes_gridpoint(sample,colvar,sigma,ndims,logweights)
        if prob[i] > max_prob:
            max_prob = prob[i]

    prob = np.reshape(prob,(num,)*ndims)

    #convert to fes
    fes=-kbt*np.log(prob/max_prob)

    return fes

collective_vars = ['tica1','tica2']
bounds = [(-1.,1.)]*len(collective_vars)

%timeit fes = compute_fes(colvar,collective_vars, bounds, num=11, kbt=2.8)


99.4 ms ± 595 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
colvar = plumed_to_pandas('data/chignolin-v3/COLVAR')
colvar = colvar.rename(columns={'deep.node-4': 'tica1','deep.node-3': 'tica2','deep.node-2': 'tica3', 'deep.node-1': 'tica4','deep.node-0': 'tica5'})
colvar = colvar[::100]
colvar

,time,rmsd_ca,end,hbonds,tica5,tica4,tica3,tica2,tica1
0,0.0,0.067477,0.500523,3.515683,0.156591,0.269741,-0.503362,-0.769360,0.860714
100,100.0,0.107275,0.548823,3.549887,-0.099905,-0.423160,-0.537948,0.000951,0.834154
200,200.0,0.055349,0.504914,3.576088,0.151266,-0.149849,0.841402,-0.592976,0.880016
300,300.0,0.058707,0.528834,3.584136,-0.104121,0.294404,-0.497918,-0.717750,0.886608
400,400.0,0.079539,0.511143,3.412412,0.161417,0.309681,0.675492,-0.644175,0.899487
...,...,...,...,...,...,...,...,...,...
524300,524300.0,0.098983,0.554914,3.546298,-0.281212,-0.084069,0.613887,0.126566,0.797860
524400,524400.0,0.101942,0.506624,3.519301,0.029760,0.076534,-0.850691,-0.024068,0.824272
524500,524500.0,0.063260,0.501542,3.545046,0.250306,0.198342,-0.532451,-0.704408,0.849083
524600,524600.0,0.297535,1.192679,1.920843,0.136975,-0.272676,0.131464,-0.207165,-0.056313


In [66]:
from stateinterpreter.MD import Loader

data_folder = "data/chignolin-v3/"

colvar_header = pd.read_csv(data_folder+'COLVAR',sep=' ',skipinitialspace=True, nrows=0).columns[2:]  

file_dict = {
    'trajectory': 'CLN025-0-protein-ALL.dcd',
    'topology' : 'CLN025-0-protein.pdb',
    'collective_vars': 'COLVAR',
    'pd.read_csv.args' : {
                        'skipinitialspace' : True, 
                        'sep' : ' ',
                        'header' : None,
                        'skiprows' : 1,
                        'comment' : '#',
                        'names' : colvar_header
                        }
}
stride = 100
data = Loader(data_folder, file_dict, stride=stride,_DEV=False)

# Rename TICA CVs
data.colvar = data.colvar.rename(columns={'deep.node-4': 'tica1','deep.node-3': 'tica2','deep.node-2': 'tica3', 'deep.node-1': 'tica4','deep.node-0': 'tica5'})

In [72]:
%timeit data.approximate_FES(collective_vars,bounds,num=21,bw_method=0.05)

88.3 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
